In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
df = pd.read_csv('new_load_data (version 2).csv')

In [ ]:
df.head()

In [ ]:
data = df[['Date','Load']]

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])

In [ ]:
stepwise_model = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=24,
                           start_P=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

In [ ]:
df = df[df["Date"]>= "2015-01-01"]
df.reset_index(inplace = True)
del df["Unnamed: 0"]

In [ ]:
del df["index"]

In [ ]:
X = df.Load
X = X.astype('float32')

y_preds = []

for i in range(24, 721, 24):
    train = X[8+i:17553+i]
    history = [x for x in train]
    model = sm.tsa.statespace.SARIMAX(history, order=(2,1,2), seasonal_order=(1,1,1,24))
    model_fit = model.fit(disp=0)
    yhat = model_fit.forecast(steps = 40)
    y_preds = y_preds+list(yhat)[16:39]
    

#Jan 2 2017 00:00:00 onwards    



In [ ]:
df_1 = pd.DataFrame(y_preds)
df_1[:6]

In [ ]:
df2 = pd.DataFrame(df.Load[17568:18258])
df2.reset_index(inplace= True)
df2 = df2['Load']
df2[:5]

In [ ]:
final_pred = [df2,df_1]
final_pred_df = pd.concat(final_pred,axis=1)
final_pred_df.head()
final_pred_df.columns=['Load',"Forecast"]
final_pred_df.tail()

In [ ]:
final_pred_df['diff'] = np.abs(final_pred_df['Load']-final_pred_df['Forecast'])
final_pred_df['a']= final_pred_df['diff']/final_pred_df['Load']
final_pred_df.head()

In [ ]:
mape_1 = []
actual = final_pred_df.Load
forecast = final_pred_df.Forecast
for i in range(0,690,24):
    mape = final_pred_df.a[i:i+24].sum()/24
    mape_1.append(mape)
print(sum(mape_1)/len(mape_1))

In [ ]:
train= data.Load[:26296]
train.tail()

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(2,1,2), seasonal_order=(1,1,1,24))
results = mod.fit()
print(results.summary())

In [ ]:
results.forecast(5000)

In [ ]:
results.forecast(steps = )